In [ ]:
!pip install selenium
!apt install chromium-chromedriver
!pip install requests-html

In [ ]:
import re
import requests
import time
from requests_html import HTML
from selenium import webdriver
from selenium.webdriver.chrome.options import Options

In [ ]:
options = webdriver.ChromeOptions()
options.add_argument('-headless')
options.add_argument('-no-sandbox')
options.add_argument('-disable-dev-shm-usage')

driver = webdriver.Chrome('chromedriver', options=options)

In [ ]:
categories = [
    "https://www.amazon.com/Best-Sellers-Toys-Games/zgbs/toys-and-games/",
    "https://www.amazon.com/Best-Sellers-Electronics/zgbs/electronics/",
    "https://www.amazon.com/best-sellers-video-games/zgbs/videogames/",
    "https://www.amazon.com/best-sellers-books/zgbs/books/"
]

In [ ]:
categories

['https://www.amazon.com/Best-Sellers-Toys-Games/zgbs/toys-and-games/',
 'https://www.amazon.com/Best-Sellers-Electronics/zgbs/electronics/',
 'https://www.amazon.com/best-sellers-video-games/zgbs/videogames/',
 'https://www.amazon.com/best-sellers-books/zgbs/books/']

In [ ]:
first_url = categories[0]
print(first_url)

https://www.amazon.com/Best-Sellers-Toys-Games/zgbs/toys-and-games/


In [ ]:
driver.get(first_url)

In [ ]:
body_el = driver.find_element_by_css_selector("body")
html_str = body_el.get_attribute("innerHTML")

In [ ]:
html_obj = HTML(html=html_str)

In [ ]:
page_links = [f"https://www.amazon.com{x}" for x in html_obj.links if x.startswith("/")]
# new_links = [x for x in new_links if "product-reviews/" not in x]

In [ ]:
page_links

['https://www.amazon.com/gp/prime/ref=nav_prime_ajax_err/144-7796612-4863140',
 'https://www.amazon.com/gp/help/customer/display.html?ie=UTF8&nodeId=468496&ref_=footer_privacy',
 'https://www.amazon.com/gp/yourstore?ref_=nav_AccountFlyout_recs',
 'https://www.amazon.com/amazon-fashion/b/?ie=UTF8&node=7141123011&ref_=nav_cs_fashion',
 'https://www.amazon.com/product-reviews/B004S8F7QM/ref=zg_bs_toys-and-games_cr_5/144-7796612-4863140?ie=UTF8&refRID=17GX9DYN7WJDEWA1YMXD',
 'https://www.amazon.com/TeeTurtle-Reversible-Octopus-Mini-Plush/dp/B07H4YQF3Q/ref=zg_bs_toys-and-games_1/144-7796612-4863140?_encoding=UTF8&psc=1&refRID=17GX9DYN7WJDEWA1YMXD',
 'https://www.amazon.com/b/?ie=UTF8&node=17728536011&ref_=sv_t_6',
 'https://www.amazon.com/hz/mycd/myx?_encoding=UTF8&ref_=footer_myk',
 'https://www.amazon.com/product-reviews/B07JB953MG/ref=zg_bs_toys-and-games_cr_18/144-7796612-4863140?ie=UTF8&refRID=17GX9DYN7WJDEWA1YMXD',
 'https://www.amazon.com/rd/uedata?tepes=1&id=17GX9DYN7WJDEWA1YMXD',
 

In [ ]:
def scrape_product_page(url, title_lookup = "#productTitle", price_lookup = "#priceblock_ourprice"):
    driver.get(url)
    time.sleep(0.5)
    body_el = driver.find_element_by_css_selector("body")
    html_str = body_el.get_attribute("innerHTML")
    html_obj = HTML(html=html_str)
    product_title = html_obj.find(title_lookup, first=True).text
    product_price = html_obj.find(price_lookup, first=True).text
    return product_title, product_price

In [ ]:
# https://www.amazon.com/LEGO-Classic-Medium-Creative-Brick/dp/B00NHQFA1I/
# https://www.amazon.com/Crayola-Washable-Watercolors-8-ea/dp/B000HHKAE2/

# <base-url>/<slug>/dp/<product_id>/

In [ ]:
my_regex_pattern = r"https://www.amazon.com/(?P<slug>[\w-]+)/dp/(?P<product_id>[\w-]+)/"
my_url = 'https://www.amazon.com/Crayola-Washable-Watercolors-8-ea/dp/B000HHKAE2/'

In [ ]:
regex = re.compile(my_regex_pattern)

In [ ]:
my_match = regex.match(my_url)
print(my_match)
print("Product ID: ", my_match['product_id'])
print("slug: ", my_match['slug'])

<_sre.SRE_Match object; span=(0, 71), match='https://www.amazon.com/Crayola-Washable-Watercolo>
Product ID:  B000HHKAE2
slug:  Crayola-Washable-Watercolors-8-ea


In [ ]:
regex_options = [
    r"https://www.amazon.com/gp/product/(?P<product_id>[\w-]+)/",
    r"https://www.amazon.com/dp/(?P<product_id>[\w-]+)/",
    r"https://www.amazon.com/(?P<slug>[\w-]+)/dp/(?P<product_id>[\w-]+)/",
]

def extract_product_id_from_url(url):
    product_id = None
    for regex_str in regex_options:
        regex = re.compile(regex_str)
        match = regex.match(url)
        if match != None:
            try:
                product_id = match['product_id']
            except:
                pass
    return product_id

In [ ]:
# page_links = [x for x in page_links if extract_product_id_from_url(x) != None]
def clean_page_links(page_links=[]):
    final_page_links = []
    for url in page_links:
        product_id = extract_product_id_from_url(url)
        if product_id != None:
            final_page_links.append({"url": url, "product_id": product_id})
    return final_page_links

cleaned_links = clean_page_links(page_links = [x.split("ref", 1)[0] for x in page_links])

In [ ]:
print(len(page_links)) # == len(cleaned_links)
page_links

198


['https://www.amazon.com/gp/prime/ref=nav_prime_ajax_err/144-7796612-4863140',
 'https://www.amazon.com/gp/help/customer/display.html?ie=UTF8&nodeId=468496&ref_=footer_privacy',
 'https://www.amazon.com/gp/yourstore?ref_=nav_AccountFlyout_recs',
 'https://www.amazon.com/amazon-fashion/b/?ie=UTF8&node=7141123011&ref_=nav_cs_fashion',
 'https://www.amazon.com/product-reviews/B004S8F7QM/ref=zg_bs_toys-and-games_cr_5/144-7796612-4863140?ie=UTF8&refRID=17GX9DYN7WJDEWA1YMXD',
 'https://www.amazon.com/TeeTurtle-Reversible-Octopus-Mini-Plush/dp/B07H4YQF3Q/ref=zg_bs_toys-and-games_1/144-7796612-4863140?_encoding=UTF8&psc=1&refRID=17GX9DYN7WJDEWA1YMXD',
 'https://www.amazon.com/b/?ie=UTF8&node=17728536011&ref_=sv_t_6',
 'https://www.amazon.com/hz/mycd/myx?_encoding=UTF8&ref_=footer_myk',
 'https://www.amazon.com/product-reviews/B07JB953MG/ref=zg_bs_toys-and-games_cr_18/144-7796612-4863140?ie=UTF8&refRID=17GX9DYN7WJDEWA1YMXD',
 'https://www.amazon.com/rd/uedata?tepes=1&id=17GX9DYN7WJDEWA1YMXD',
 

In [ ]:
print(len(cleaned_links))
cleaned_links

49


[{'product_id': 'B07H4YQF3Q',
  'url': 'https://www.amazon.com/TeeTurtle-Reversible-Octopus-Mini-Plush/dp/B07H4YQF3Q/'},
 {'product_id': 'B013RQPB5C',
  'url': 'https://www.amazon.com/Crayola-Ultra-Clean-Washable-Non-Toxic-Classrooms/dp/B013RQPB5C/'},
 {'product_id': 'B07F7KJNKK',
  'url': 'https://www.amazon.com/Fidget-Sensory-Anti-Anxiety-Squeeze-Soybean/dp/B07F7KJNKK/'},
 {'product_id': 'B07F81VDL2',
  'url': 'https://www.amazon.com/Relieves-Children-Assortment-Birthday-Classroom/dp/B07F81VDL2/'},
 {'product_id': 'B08LB3HB31',
  'url': 'https://www.amazon.com/Bubble-Sensory-Special-Reliever-Squeeze/dp/B08LB3HB31/'},
 {'product_id': 'B0037710BG',
  'url': 'https://www.amazon.com/Play-Doh-Party-Bag-Dough-Count/dp/B0037710BG/'},
 {'product_id': 'B085KTB1WY',
  'url': 'https://www.amazon.com/LeapFrog-Animals-Book-Frustration-Packaging/dp/B085KTB1WY/'},
 {'product_id': 'B00JM5GW10',
  'url': 'https://www.amazon.com/Play-Doh-Modeling-Compound-Non-Toxic-Exclusive/dp/B00JM5GW10/'},
 {'produ

In [ ]:
def perform_scrape(cleaned_items=[]):
    data_extracted = []
    i = 0
    for obj in cleaned_items:
        link = obj['url']
        product_id = obj['product_id']
        title, price = (None, None)
        try:
            title, price = scrape_product_page(link)
        except:
            pass
        if title != None and price != None:
            #print(link, title, price)
            i += 1
            print(str(i)+") "+str(title))
            print("Price: " + str(price))
            print("Link: " + str(link)+"\n")
            product_data = {
                "url": link,
                "product_id": product_id,
                "title": title,
                "price": price
            }
            data_extracted.append(product_data)
    return data_extracted

In [ ]:
extracted_data = perform_scrape(cleaned_items=cleaned_links)

1) The Original Reversible Octopus Plushie | TeeTurtle’s Patented Design | Black and Gray | Show your mood without saying a word!
Price: $15.00
Link: https://www.amazon.com/TeeTurtle-Reversible-Octopus-Mini-Plush/dp/B07H4YQF3Q/

2) Crayola Ultra Clean Washable Markers, Kids Indoor Activities At Home, Broad Line, 40 Classic Colors
Price: $12.22
Link: https://www.amazon.com/Crayola-Ultra-Clean-Washable-Non-Toxic-Classrooms/dp/B013RQPB5C/

3) Push pop pop Bubble Sensory Fidget Toy Autism Special Needs Stress Reliever, Squeeze Sensory Toy Great for The Old and The Young (Green)
Price: $6.00
Link: https://www.amazon.com/Bubble-Sensory-Special-Reliever-Squeeze/dp/B08LB3HB31/

4) Play-Doh Handout Non-Toxic Modeling Compound 15 1 oz cans - Stocking stuffer
Price: $4.99
Link: https://www.amazon.com/Play-Doh-Party-Bag-Dough-Count/dp/B0037710BG/

5) LeapFrog 100 Animals Book (Frustration Free Packaging), Pink
Price: $17.99
Link: https://www.amazon.com/LeapFrog-Animals-Book-Frustration-Packaging/d

In [ ]:
import json
### print json  (global helper)
def print_json(js):
    print(json.dumps(js, indent=2))
for data in extracted_data:
    print_json(data)

{
  "url": "https://www.amazon.com/TeeTurtle-Reversible-Octopus-Mini-Plush/dp/B07H4YQF3Q/",
  "product_id": "B07H4YQF3Q",
  "title": "The Original Reversible Octopus Plushie | TeeTurtle\u2019s Patented Design | Black and Gray | Show your mood without saying a word!",
  "price": "$15.00"
}
{
  "url": "https://www.amazon.com/Crayola-Ultra-Clean-Washable-Non-Toxic-Classrooms/dp/B013RQPB5C/",
  "product_id": "B013RQPB5C",
  "title": "Crayola Ultra Clean Washable Markers, Kids Indoor Activities At Home, Broad Line, 40 Classic Colors",
  "price": "$12.22"
}
{
  "url": "https://www.amazon.com/Bubble-Sensory-Special-Reliever-Squeeze/dp/B08LB3HB31/",
  "product_id": "B08LB3HB31",
  "title": "Push pop pop Bubble Sensory Fidget Toy Autism Special Needs Stress Reliever, Squeeze Sensory Toy Great for The Old and The Young (Green)",
  "price": "$6.00"
}
{
  "url": "https://www.amazon.com/Play-Doh-Party-Bag-Dough-Count/dp/B0037710BG/",
  "product_id": "B0037710BG",
  "title": "Play-Doh Handout Non-To